## Setup

In [ ]:
# ---------------------------------------------------
# 📦 LIBRERÍAS
# ---------------------------------------------------
import os
import time
import json
import gspread
import requests
from io import BytesIO
from PIL import Image
from google.api_core import exceptions
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google import genai
from google.genai.types import Part, GenerateContentConfig

PROJECT_ID_IMAGE = "lmes-peoplechatbot-p-mzd"
PROJECT_ID_MEASURES = "lmes-peoplechatbot-p-mzd"
LOCATION_IMAGE = "global"
LOCATION_MEASURES = "europe-west1"

In [2]:

# Usa las credenciales configuradas con gcloud auth application-default login
# Los scopes ya están incluidos en las credenciales ADC
creds, _ = google.auth.default()
drive_service = build('drive', 'v3', credentials=creds)
gc = gspread.authorize(creds)

In [8]:
# ---------------------------------------------------
# ⚙️ PARÁMETROS PRINCIPALES
# ---------------------------------------------------
SHEET_ID = "1EUPvd9gDQ5TjoboTdqAiW2mOChAyE7ysc-DkedaK8Cg"
WORKSHEET_NAME = "Sheet1"
OUTPUT_FOLDER_NAME = "COTAS_TEST"
BATCH_SIZE = 50

# --- NUEVOS PARÁMETROS PARA REINTENTOS ---
RETRY_ATTEMPTS = 3
RETRY_DELAY_SECONDS = 5

AI_MODEL_NAME = "gemini-2.5-flash-image-preview"
PRICE_PER_IMAGE = 0.039

MEASURES_MODEL_NAME = "gemini-2.0-flash"
PRICE_PER_MILLION_TOKENS = 0.00035

# --- PROMPTS (sin cambios) ---
NO_COTAS_PROMPT_TEMPLATE = """
{{
  "task_description": "Based on the provided reference image, create a single, highly detailed, black and white line art illustration of the product '{title}'. The output must be a pure visual representation without any annotations, text, or dimensions.",
  "style_definition": {{"primary_style": "professional technical drawing", "secondary_style": "architectural blueprint", "rendering_quality": "hyperrealistic_detail", "color_palette": "black_and_white_monochrome", "line_work": "clean, crisp, precise, and well-defined lines", "shading": "none (strictly no fills, no gradients, no shadows)"}},
  "technical_specifications": {{"resolution": "high-definition, print_quality", "rendering": "maximum_anti-aliasing, no_noise_artifacts", "perspective": "maintain the exact perspective, angle, and proportions from the reference image"}},
  "content_rules": {{"subject": "Depict the product ONLY. Trace the reference image to capture its exact solid form, layered structure, and all visible material properties and textures (e.g., wood grain, metal finishes, small grooves, screws).", "background": "pure_white, completely empty, no environmental elements", "annotations": "strictly_forbidden (absolutely no dimensions, no labels, no text, no arrows, no measurements of any kind)"}},
  "output_specifications": {{"composition": "Ensure the entire product illustration is centered and fully visible within the canvas.", "margins": "Provide a small, consistent white margin (approximately 5-10%) around the product to prevent cropping."}}
}}
"""
CON_COTAS_PROMPT_BASE = """
--- CORE INSTRUCTION ---
Your goal is to create a highly detailed, black and white line art illustration of the product '{title}' based on the provided reference image, and then accurately overlay precise dimension lines and labels as specified in the "ADD DIMENSIONS" section.
--- STYLE GUIDELINES (Base Illustration) ---
1.  **Line Art:** Generate a clean, crisp, black and white line drawing for the product. No colors, no shadows, no fills, no gradients.
2.  **Detail Level:** Emphasize an extremely high level of detail. Trace the reference image as if converting a photograph into a precise technical drawing.
3.  **Perspective:** Maintain the exact perspective and angle of the object as seen in the reference image.
--- CONTENT GUIDELINES (Base Illustration) ---
1.  **Product Only:** The illustration must only depict the product itself.
--- OUTPUT GUIDELINES (Base Illustration) ---
1.  **Canvas Containment:** Ensure the entire product illustration is fully contained within the image canvas, leaving ample space for dimensions.
"""
ADD_DIMENSIONS_TEMPLATE = """
--- ADD DIMENSIONS ---
Now, add clear and professionally styled dimension lines to the illustration based on these specifications.
- Add a dimension line for the **Width** labeled as: **{width}**
- Add a dimension line for the **Height** labeled as: **{height}**
- Add a dimension line for the **Depth** labeled as: **{depth}**
If a dimension is 'Not specified', do not draw a line or a label for it.
"""
MEASURES_PROMPT_TEMPLATE = """
Given the following attributes of a product, extract the primary overall dimensions (width, height, depth). Return the output ONLY as a valid JSON object.
Attributes: {atributos}
Rules:
- Identify primary overall width, height, and depth.
- If a dimension is not found, its value should be "Not specified".
- Format the value as a string with unit, e.g., "245 cm". Convert mm to cm.
- Output must be a valid JSON object and nothing else.
Example 1: {{"width": "245 cm", "height": "245 cm", "depth": "300 cm"}}
"""
genai_client = genai.Client(vertexai=True, project=PROJECT_ID_IMAGE, location=LOCATION_IMAGE, credentials=creds)

In [9]:
# ---------------------------------------------------
# 🛠️ FUNCIONES AUXILIARES
# ---------------------------------------------------
def get_drive_folder_id(folder_name):
    query = f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed=false"
    response = drive_service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    folders = response.get('files', [])
    return folders[0]['id'] if folders else None

def get_or_create_drive_subfolder(subfolder_name, parent_folder_id):
    print(f"   Buscando subcarpeta '{subfolder_name}'...")
    query = f"mimeType='application/vnd.google-apps.folder' and name='{subfolder_name}' and '{parent_folder_id}' in parents and trashed=false"
    response = drive_service.files().list(q=query, spaces='drive', fields='files(id)').execute()
    folders = response.get('files', [])
    if folders:
        print(f"   ✅ Subcarpeta encontrada.")
        return folders[0]['id']
    else:
        print(f"   ⚠️ Subcarpeta no encontrada. Creando nueva carpeta...")
        file_metadata = {'name': subfolder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [parent_folder_id]}
        folder = drive_service.files().create(body=file_metadata, fields='id').execute()
        print(f"   ✅ Subcarpeta creada.")
        return folder.get('id')

def upload_to_drive(local_path, file_name, folder_id):
    file_metadata = {"name": file_name, "parents": [folder_id]}
    media = MediaFileUpload(local_path, mimetype="image/png")
    file = drive_service.files().create(body=file_metadata, media_body=media, fields="id, webViewLink").execute()
    return file.get("webViewLink")

def download_image_from_url(url):
    try:
        response = requests.get(url, timeout=20)
        response.raise_for_status()
        image_bytes = response.content
        image = Image.open(BytesIO(image_bytes))
        mime_type = "image/jpeg" if image.format.lower() in ["jpeg", "jpg"] else "image/png"
        return image_bytes, mime_type
    except Exception as e:
        print(f"       ⚠️ Error al descargar imagen {url}: {e}")
        return None, None

# --- NUEVA FUNCIÓN CON LÓGICA DE REINTENTOS ---
def generate_image_with_retries(prompt_text, image_bytes, mime_type, config):
    """Genera una imagen, reintentando si la API falla o devuelve una respuesta vacía."""
    contents = [Part.from_text(text=prompt_text), Part.from_bytes(data=image_bytes, mime_type=mime_type)]

    for attempt in range(RETRY_ATTEMPTS):
        try:
            response = genai_client.models.generate_content(
                model=AI_MODEL_NAME,
                contents=contents,
                config=config
            )

            # Validar la respuesta
            if (response.candidates and response.candidates[0].content.parts and response.candidates[0].content.parts[0].inline_data):
                return response # Éxito, devuelve la respuesta completa
            else:
                print(f"      ⚠️ Intento {attempt + 1} fallido: Respuesta de API inválida (posiblemente bloqueada).")

        except Exception as e:
            print(f"      ⚠️ Intento {attempt + 1} fallido con error de API: {e}")

        # Si no es el último intento, esperar antes de reintentar
        if attempt < RETRY_ATTEMPTS - 1:
            print(f"      ⌛ Esperando {RETRY_DELAY_SECONDS} segundos para reintentar...")
            time.sleep(RETRY_DELAY_SECONDS)

    return None # Si todos los intentos fallan, devuelve None

In [ ]:
# ---------------------------------------------------
# 🚀 PROCESO PRINCIPAL
# ---------------------------------------------------
total_input_tokens = 0
total_output_tokens = 0
images_sin_cotas_generated = 0
images_con_cotas_generated = 0
total_cost = 0.0

print("\n🚀 Iniciando proceso de generación de diagramas técnicos...")
try:
    parent_folder_id = get_drive_folder_id(OUTPUT_FOLDER_NAME)
    print(parent_folder_id)
    if not parent_folder_id: raise Exception(f"Carpeta principal '{OUTPUT_FOLDER_NAME}' no encontrada.")

    subfolder_name = input("➡️ Introduce el nombre de la carpeta para esta ejecución (ej: lote_octubre_07): ")
    if not subfolder_name:
        subfolder_name = f"ejecucion_{time.strftime('%Y%m%d_%H%M%S')}"
        print(f"   Nombre vacío, se usará uno por defecto: '{subfolder_name}'")

    final_output_folder_id = get_or_create_drive_subfolder(subfolder_name, parent_folder_id)

    sheet = gc.open_by_key(SHEET_ID).worksheet(WORKSHEET_NAME)
    data = sheet.get_all_values()
    headers = data[0]
    rows = data[1:]

    if "Medidas IA" not in headers: headers.append("Medidas IA")
    if "Url picto REALIZADO CON IA" not in headers: headers.append("Url picto REALIZADO CON IA")
    if "Url picto-cotas REALIZADO CON IA" not in headers: headers.append("Url picto-cotas REALIZADO CON IA")
    sheet.update(range_name="A1", values=[headers])
    print(f"\n✅ Setup completado. Se encontraron {len(rows)} filas. Las imágenes se guardarán en '{subfolder_name}'.")

except Exception as e:
    print(f"❌ Error en configuración inicial: {e}")
    exit()

all_jobs = [{"sheet_index": i + 2, "referencia": row[0], "titulo": row[2], "atributos": row[3], "url": row[1]} for i, row in enumerate(rows) if len(row) >= 4]

if not all_jobs:
    print("\n❌ No hay trabajos válidos para procesar. Finalizando.")
    exit()

config_images = GenerateContentConfig(
    safety_settings=[
        {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'threshold': 'BLOCK_NONE'},
        {'category': 'HARM_CATEGORY_HARASSMENT', 'threshold': 'BLOCK_NONE'},
        {'category': 'HARM_CATEGORY_HATE_SPEECH', 'threshold': 'BLOCK_NONE'},
        {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'threshold': 'BLOCK_NONE'},
    ]
)

model_measures = genai.Client(
    vertexai=True, location=LOCATION_MEASURES, project=PROJECT_ID_MEASURES
)

total_lotes = (len(all_jobs) + BATCH_SIZE - 1) // BATCH_SIZE

for i in range(0, len(all_jobs), BATCH_SIZE):
    current_batch_jobs = all_jobs[i:i + BATCH_SIZE]
    lote_num = (i // BATCH_SIZE) + 1

    print(f"\n--- Procesando Lote {lote_num} / {total_lotes} (Productos {i+1} a {i+len(current_batch_jobs)}) ---")

    updates_for_this_batch = []
    for job in current_batch_jobs:
        print(f"\n  Procesando: {job['referencia']} (Fila {job['sheet_index']})")

        try:
            # 1. Extraer Medidas

            # Esto hay que cambiar a una consulta
            print("    📏 Extrayendo medidas...")
            prompt_medidas = MEASURES_PROMPT_TEMPLATE.format(atributos=job['atributos'])
            response_medidas = model_measures.models.generate_content(model=MEASURES_MODEL_NAME, contents=prompt_medidas)

            total_input_tokens += response_medidas.usage_metadata.prompt_token_count
            total_output_tokens += response_medidas.usage_metadata.candidates_token_count

            response_text = response_medidas.text.strip().replace("```json", "").replace("```", "")
            measures_dict = json.loads(response_text)
            parts = [f"{k.capitalize()}: {v}" for k, v in measures_dict.items() if v != "Not specified"]
            medidas_ia_string = ". ".join(parts) + "." if parts else "Measures not specified."
            updates_for_this_batch.append(gspread.Cell(job['sheet_index'], headers.index("Medidas IA") + 1, medidas_ia_string))

            # 2. Descargar imagen
            print("    📥 Descargando imagen de referencia...")
            image_bytes, mime_type = download_image_from_url(job['url'])
            if not image_bytes:
                print("    ❌ No se pudo descargar la imagen. Saltando generación para este producto.")
                continue

            # 3. Generar imagen SIN cotas (con reintentos)
            print("    🎨 Generando imagen SIN cotas...")
            prompt_sin_cotas = NO_COTAS_PROMPT_TEMPLATE.format(title=job['titulo'])
            response_img_sin_cotas = generate_image_with_retries(prompt_sin_cotas, image_bytes, mime_type, config_images)

            if not response_img_sin_cotas:
                raise ValueError("No se recibió una imagen SIN cotas válida después de varios intentos.")

            img_bytes_sin_cotas = response_img_sin_cotas.candidates[0].content.parts[0].inline_data.data
            local_path = f"/tmp/{job['referencia']}_sin_cotas.png"
            with open(local_path, "wb") as f: f.write(img_bytes_sin_cotas)
            drive_url = upload_to_drive(local_path, f"{job['referencia']}_pictograma.png", final_output_folder_id)
            updates_for_this_batch.append(gspread.Cell(job['sheet_index'], headers.index("Url picto REALIZADO CON IA") + 1, drive_url))
            images_sin_cotas_generated += 1
            total_cost += PRICE_PER_IMAGE
            os.remove(local_path)
            print("    ✅ Imagen SIN cotas generada y subida.")

            # 4. Generar imagen CON cotas (con reintentos)
            if any(v != "Not specified" for v in measures_dict.values()):
                print("    📐 Generando imagen CON cotas...")
                dimension_text = ADD_DIMENSIONS_TEMPLATE.format(**measures_dict)
                prompt_con_cotas = CON_COTAS_PROMPT_BASE.format(title=job['titulo']) + dimension_text
                response_img_con_cotas = generate_image_with_retries(prompt_con_cotas, image_bytes, mime_type, config_images)

                if not response_img_con_cotas:
                    raise ValueError("No se recibió una imagen CON cotas válida después de varios intentos.")

                img_bytes_con_cotas = response_img_con_cotas.candidates[0].content.parts[0].inline_data.data
                local_path = f"/tmp/{job['referencia']}_con_cotas.png"
                with open(local_path, "wb") as f: f.write(img_bytes_con_cotas)
                drive_url = upload_to_drive(local_path, f"{job['referencia']}_pictograma-cotas.png", final_output_folder_id)
                updates_for_this_batch.append(gspread.Cell(job['sheet_index'], headers.index("Url picto-cotas REALIZADO CON IA") + 1, drive_url))
                images_con_cotas_generated += 1
                total_cost += PRICE_PER_IMAGE
                os.remove(local_path)
                print("    ✅ Imagen CON cotas generada y subida.")

        except Exception as e:
            print(f"    ❌❌ ERROR FATAL procesando {job['referencia']}: {e}")

    if updates_for_this_batch:
        print(f"\n   ⚡ Actualizando {len(updates_for_this_batch)} celdas en Google Sheets para el Lote {lote_num}...")
        sheet.update_cells(updates_for_this_batch, value_input_option='USER_ENTERED')
        print(f"   ✅ Lote {lote_num} completado y hoja actualizada.")

    if lote_num < total_lotes:
      print("\n   ⌛ Pausa de 5 segundos antes del siguiente lote...")
      time.sleep(5)


🚀 Iniciando proceso de generación de diagramas técnicos...
1FetU3MiLK0i49zGNgZdiBYITQ6OrI06E
   Buscando subcarpeta 'lote_octubre_07'...
   ✅ Subcarpeta encontrada.

✅ Setup completado. Se encontraron 1643 filas. Las imágenes se guardarán en 'lote_octubre_07'.

--- Procesando Lote 1 / 33 (Productos 1 a 50) ---

  Procesando: 14059360 (Fila 2)
    📏 Extrayendo medidas...
    📥 Descargando imagen de referencia...
    🎨 Generando imagen SIN cotas...
      ⚠️ Intento 1 fallido: Respuesta de API inválida (posiblemente bloqueada).
      ⌛ Esperando 5 segundos para reintentar...
    ✅ Imagen SIN cotas generada y subida.
    📐 Generando imagen CON cotas...
    ✅ Imagen CON cotas generada y subida.

  Procesando: 14059374 (Fila 3)
    📏 Extrayendo medidas...
    📥 Descargando imagen de referencia...
    🎨 Generando imagen SIN cotas...


KeyboardInterrupt: 

In [ ]:
# --- CÁLCULO FINAL DE COSTES ---
cost_text = (total_input_tokens + total_output_tokens) / 1_000_000 * PRICE_PER_MILLION_TOKENS
cost_images = (images_sin_cotas_generated + images_con_cotas_generated) * PRICE_PER_IMAGE
total_cost = cost_text + cost_images

print("\n" + "="*50)
print("🎉 PROCESO FINALIZADO 🎉")
print(f"     - Imágenes SIN cotas generadas: {images_sin_cotas_generated}")
print(f"     - Imágenes CON cotas generadas: {images_con_cotas_generated}")
print(f"     - Tokens de entrada (texto): {total_input_tokens}")
print(f"     - Tokens de salida (texto): {total_output_tokens}")
print(f"     - Coste estimado de texto: ${cost_text:.6f}")
print(f"     - Coste estimado de imágenes: ${cost_images:.6f}")
print(f"     - COSTE TOTAL ESTIMADO: ${total_cost:.6f}")
print("="*50)